In [ ]:
%run ../../Week_2/07.15-abalone/AI_abalone.ipynb

## A.1 메인함수 정의

In [ ]:
def steel_exec(epoch_count=10, mb_size = 10, report =1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count,mb_size,report)

## B.1 데이터 적재 함수 정의

In [ ]:
def load_steel_dataset():
    with open('./faults.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    
    input_cnt, output_cnt = 27, 7 # 선택분류에 알맞다록 출력 벡터 크기를 분류 항목 수인 7로 지정   
    data = np.asarray(rows, dtype='float32') # 연선을 위하여 리스트 형태인 rows를 배열 형태로 변환

## D.2 E.2 후처리 과정에 대한 순전파와 역전파 함수의 재정의 

In [ ]:
# 손실함수를 계산해 순전파 과정을 마무리 짓는 함수
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output) # 신경망의 손실함수 계산
    loss = np.mean(entropy) # 범주별 entropy 값에 대한 평균
    
    return loss, [y, output, entropy] # loss 반환 및 역전파에 사용될 보조정보 반환


# 신경망에 대한 출력에 대한 손실 기울기를 계산해 역전파 과정을 시작하는 함수
def backprop_postproc(G_loss, aux): 
    y, output, entropy = aux # 리스트 타입의 보조정보를 각 변수로 독립
    g_loss_entropy = 1.0 / np.prod(entropy.shape) # dL/dL와 G_entropy 사이의 부분 기울기
    # soft_cross_entropy_with_logits_derv()는 소프트맥스 교차 엔트로피의 편미분 과정
    # 이는 실제 정답에 의해 표현되는 확률분포 p, 즉 y값과 신경망에 의한 추정확률분포 q, 즉 output을 활용하여 손실기울기를 구하는 함수
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y,output) 

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output # Y에 대한 손실함수의 편미분

## D.3 정확도 계산 함수 정의

In [ ]:
def eval_accuracy(output, y):
    # 신경망 추정에서의 선택 결과는 np.argmax()를 이용해 후보 항목에 관한 각 로짓값을 담고 있는 output 벡터에서 가장 큰 값이 어디에 있는지 조사 
    # 중심선 기준을 1, 즉 y축으로 맞춰주어 신경망이 출력한 값 중 가장 큰 값의 인덱스를 반환 및 그 값을 estimate(추정 결과)라는 객체에 할당 및 저장
    # 이 때 확률 분포로 변환해도 로짓값의 대소관계는 그대로 유지 -> 굳이 소프트맥스 함수를 불러 로짓값 벡터 확률 분포로 변환할 필요가 없음
    estimate = np.argmax(output, axis=1) 
    # 정답 데이터에 대해 가장 큰 값을 answer라는 객체에 넣어 신경망의 추정치와 비교할 수 있도록 준비
    answer = np.argmax(y, axis=1)
    # 서로 비교하며 불리안 타입으로 결과를 저장
    correct = np.equal(estimate, answer)
    
    # 결과값과 추정치의 선택 일치 여부를 따져 일치한 비율을 정확도로 보고해 주도록 함
    return np.mean(correct)  

## F.1~3 소프트맥스 관련 함수 정의

In [ ]:
# 소프트맥스와 관련된 함수들을 정의하는 순서 (소프트맥스 변형식)

# 데이터에 담긴 여러 데이터에 댈한 벡터들을 담은 행렬을 처리 대상
def softmax(x): 
    max_elem = np.max(x, axis=1) # 소프트맥스 변형식에 따라 입력으로 드렁올 각 연산결과에 대해 가장 높은 값을 선택하여 저장 (axis=1 : 열 기준)
    # i번째 x와 가장 큰 x값을 빼주는 과정
    # transpose()를 적용한 이유는 Numpy의 행렬 연산에 있어, 위에서 도출한 max_elem이 현재 열을 기준으로 정렬되어 있는데
    # 이 때, 열에 대한 반복 감산을 진행하기 위해서 기존 x값을 행렬 전환 시켜준 후 감산을 진행하고 다시 행렬 전환으로 원상 복귀
    diff = (x.transpose() - max_elem).transpose() 
    # 위와 같이 구한 차이 diff에 지수 함수를 취함
    exp = np.exp(diff) 
    sum_exp = np.sum(exp, axis=1) # np.sum()을 통해 모든 값을 합산 / 이 부분은 분모가 되는 부분
    # 소프트맥스 변형식처럼 식을 구하기 위해 분모와 분자를 위치 / 이전과 같은 이유로 행렬 전환을 두 번 진행
    probs = (exp.transpose()/sum_exp).transpose() # probs : 확률 벡터 

    return probs  


# 소프트맥스 교차 엔트로피 변형식 구현 부분
def softmax_cross_entropy_with_logits(labels, logits):
    # 퍼셉트론 연산을 통과한 신경망의 추정치 로짓값을 소프트맥스 함수로 통과시켜 각 확률분포 값으로 저장
    probs = softmax(logits) 

    # 신경망의 추정치에 대한 확률 분포 값에 계산 폭주를 막기 위한 방법으로 아주 작은 엡실론 값을 더해주도록 함
    return -np.sum(labels * np.log(probs + 1.0e-10), axis=1) # 1.0e-10 : 임의의 엡실론 값
    

# 소프트맥스 교차 엔트로피 편미분 부분
def softmax_cross_entropy_with_logits_derv(labels, logits):
    # 신경망이 추정한 확률 분포 값들에 단순히 정답에 의한 확률 분포 값들을 감산해주는 것만으로 간단히 편미분을 구해낼 수 있게 된다.
    # (신경망 백터에 소프트맥스 함수를 적용한 확률 분포 Q와 정답 확률 분포 벡터의 확률 분포 p에서 유도한 도함수 참고)
    return softmax(logits) - labels

## 실행하기

In [ ]:
steel_exec() # 0.455

In [ ]:
LEARNING_RATE = 0.0001
steel_exec() # 0.189

이렇게 낮은 품질의 신경망 모델에 대한 주 원인으로는 차원에 비해 부족한 대이터의 양이 자주 꼽힌다.  
또한, 이 모델은 은닉층이 없는 단층 퍼셉트론이므로 생각보다 신경망의 구조가 너무 단순해서 일 수도 있다.